## Slicing and Dicing Our Data

For this example, we'll utilize the Drug Review Dataset, which is hosted on the UC Irvine Machine Learning Repository and includes patient ratings on various drugs, as well as the ailment being treated and a 10-star satisfaction rating.

To begin, we must download and extract the data using the wget and unzip commands, respectively:

In [1]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
!unzip drugsCom_raw.zip

--2024-09-26 15:54:36--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip        [   <=>              ]  41.00M  69.4MB/s    in 0.6s    

2024-09-26 15:54:36 (69.4 MB/s) - ‘drugsCom_raw.zip’ saved [42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [2]:
from datasets import load_dataset

In [3]:
data_files = {
    "train":"/content/drugsComTrain_raw.tsv",
    "test":"/content/drugsComTest_raw.tsv",

}

In [4]:
# Load the dataset using the 'csv' format
dataset = load_dataset(
    "csv",
    data_files=data_files,
    delimiter="\t"
)

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [6]:
# Shuffle the training data and select a subset of 1000 samples from it
sample = dataset["train"].shuffle(seed=42).select(range(1000))
sample[:3]

{'Unnamed: 0': [87571, 178045, 80482],
 'drugName': ['Naproxen', 'Duloxetine', 'Mobic'],
 'condition': ['Gout, Acute', 'ibromyalgia', 'Inflammatory Conditions'],
 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."',
  '"I have been taking Mobic for over a year with no side effects other than 

In [7]:
for key in dataset.keys():
  assert len(dataset[key]) == len(dataset[key].unique("Unnamed: 0"))

In [8]:
# Rename a column in the dataset

dataset = dataset.rename_column(
    original_column_name="Unnamed: 0", new_column_name="patient_id")

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [10]:
def lowercase_function(sample):
  # Convert the "condition" value for the sample to lowercase
  return {'condition': sample["condition"].lower()}

In [11]:
dataset = dataset.filter(lambda x: x["condition"] is not None)
dataset = dataset.map(lowercase_function)

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

In [12]:
dataset["train"]["condition"][:3]

['left ventricular dysfunction', 'adhd', 'birth control']

## Creating new columns

In [13]:
def review_length(sample):
  return {"review_length": len(sample["review"].split())}

In [14]:
dataset = dataset.map(review_length)
dataset["train"][0]

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

{'patient_id': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review_length': 17}

In [15]:
dataset["train"].sort("review_length")[:3]

{'patient_id': [111469, 13653, 53602],
 'drugName': ['Ledipasvir / sofosbuvir',
  'Amphetamine / dextroamphetamine',
  'Alesse'],
 'condition': ['hepatitis c', 'adhd', 'birth control'],
 'review': ['"Headache"', '"Great"', '"Awesome"'],
 'rating': [10.0, 10.0, 10.0],
 'date': ['February 3, 2015', 'October 20, 2009', 'November 23, 2015'],
 'usefulCount': [41, 3, 0],
 'review_length': [1, 1, 1]}

In [16]:
dataset = dataset.filter(lambda x: x["review_length"] > 30)
dataset.num_rows

Filter:   0%|          | 0/160398 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53471 [00:00<?, ? examples/s]

{'train': 138514, 'test': 46108}

In [17]:
import html
dataset = dataset.map(lambda x: {"review": html.unescape(x["review"])})

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [18]:
from transformers import AutoTokenizer

In [19]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(sample):
  return tokenizer(sample["review"], truncation=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
%time toknized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

CPU times: user 1min 34s, sys: 837 ms, total: 1min 35s
Wall time: 1min 5s


In [21]:
%time toknized_dataset = dataset.map(tokenize_function, batched=True, num_proc=8)

Map (num_proc=8):   0%|          | 0/138514 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/46108 [00:00<?, ? examples/s]

CPU times: user 1.05 s, sys: 321 ms, total: 1.37 s
Wall time: 1min 10s


In [22]:
%time toknized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/138514 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/46108 [00:00<?, ? examples/s]

CPU times: user 1.02 s, sys: 217 ms, total: 1.23 s
Wall time: 1min 7s


In [23]:
# Tokenizes the 'review' field from the sample and splits it into tokens
def tokenize_and_split(sample):
  return tokenizer(
                      sample["review"],
                      truncation=True,
                      max_length=128,
                      return_overflowing_tokens=True
                  )

In [24]:
results = dataset.map(tokenize_and_split, batched=True)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

ArrowInvalid: Column 8 named input_ids expected length 1000 but got length 1463

In [25]:
dataset["train"].column_names

['patient_id',
 'drugName',
 'condition',
 'review',
 'rating',
 'date',
 'usefulCount',
 'review_length']

In [26]:
# Apply the tokenize_and_split function to the entire dataset in a batched manner.
tokenized_dataset = dataset.map(
    tokenize_and_split,
    batched=True,
    # Remove original columns after tokenization to keep only the necessary features.
    remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [27]:
tokenized_dataset["train"].column_names

['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping']

In [28]:
len(tokenized_dataset["train"]), len(dataset["train"])

(206772, 138514)

In [29]:
def tokenize_and_split(sample):
  # Tokenize the review text, truncating to max length and allowing overflow
  result = tokenizer(
                      sample["review"],
                      truncation=True,
                      max_length=128,
                      return_overflowing_tokens=True
                  )

  mapping = result.pop("overflow_to_sample_mapping")

  # Map additional sample data using the overflow mapping
  for key, value in sample.items():
    result[key] = [value[i] for i in mapping]

  return result

In [30]:
tokenized_dataset = dataset.map(
    tokenize_and_split,
    batched=True,
)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [31]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 206772
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 68876
    })
})

## From Datasets to DataFrames and Back

Let's convert our dataset to `Pandas DataFrame`.

In [32]:
dataset.set_format('pandas')

In [33]:
dataset["train"][:5]

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141
1,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134
2,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,89
3,35696,Buprenorphine / naloxone,opiate dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,124
4,155963,Cialis,benign prostatic hyperplasia,"""2nd day on 5mg started to work with rock hard...",2.0,"November 28, 2015",43,68


In [34]:
train_df = dataset["train"][:]

In [35]:
train_df.head()

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141
1,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134
2,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,89
3,35696,Buprenorphine / naloxone,opiate dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,124
4,155963,Cialis,benign prostatic hyperplasia,"""2nd day on 5mg started to work with rock hard...",2.0,"November 28, 2015",43,68


In [36]:
frequesncies = (
    train_df["condition"].value_counts().to_frame().reset_index()
    .rename(columns = {"index": "condition", "count":"frequency"})
)

In [37]:
frequesncies.head()

,condition,frequency
0,birth control,27655
1,depression,8023
2,acne,5209
3,anxiety,4991
4,pain,4744


In [38]:
from datasets import Dataset

In [39]:
freq_dataset = Dataset.from_pandas(frequesncies)
freq_dataset

Dataset({
    features: ['condition', 'frequency'],
    num_rows: 819
})

In [40]:
dataset.reset_format()

## Creating a Validation Set

In [41]:
dataset_processed = dataset["train"].train_test_split(train_size=0.8, seed=42)

In [42]:
dataset_processed

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
})

In [43]:
dataset_processed["validation"] = dataset_processed.pop("test")
dataset_processed["test"] = dataset["test"]

In [44]:
dataset_processed

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

## Saving the Dataset

In [45]:
dataset_processed.save_to_disk("drug-reviews")

Saving the dataset (0/1 shards):   0%|          | 0/110811 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/27703 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/46108 [00:00<?, ? examples/s]

In [46]:
from datasets import load_from_disk

In [47]:
drug_dataset = load_from_disk("drug-reviews")

In [48]:
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

### Saving and Loading Dataset in JSONL Format

In [49]:
for split, dataset in dataset_processed.items():
  dataset.to_json(f"drug-review-{split}.jsonl")

Creating json from Arrow format:   0%|          | 0/111 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/28 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/47 [00:00<?, ?ba/s]

In [50]:
!head -n 1 drug-review-train.jsonl

{"patient_id":89879,"drugName":"Cyclosporine","condition":"keratoconjunctivitis sicca","review":"\"I have used Restasis for about a year now and have seen almost no progress.  For most of my life I've had red and bothersome eyes. After trying various eye drops, my doctor recommended Restasis.  He said it typically takes 3 to 6 months for it to really kick in but it never did kick in.  When I put the drops in it burns my eyes for the first 30 - 40 minutes.  I've talked with my doctor about this and he said it is normal but should go away after some time, but it hasn't. Every year around spring time my eyes get terrible irritated  and this year has been the same (maybe even worse than other years) even though I've been using Restasis for a year now. The only difference I notice was for the first couple weeks, but now I'm ready to move on.\"","rating":2.0,"date":"April 20, 2013","usefulCount":69,"review_length":147}


In [51]:
data_files = {
    "train": "drug-review-train.jsonl",
    "test": "drug-review-test.jsonl",
    "validation": "drug-review-validation.jsonl"
}

drug_dataset = load_dataset("json", data_files=data_files)
drug_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
})